In [ ]:
!pip install -e "git+https://github.com/bcollazo/catanatron.git@master#egg=catanatron&subdirectory=catanatron_core"
!pip install -e "git+https://github.com/bcollazo/catanatron.git@master#egg=catanatron_gym&subdirectory=catanatron_gym"
!pip install pandas
!pip install matplotlib
!pip install -e "git+https://github.com/bcollazo/catanatron.git@master#egg=catanatron_experimental&subdirectory=catanatron_experimental"

In [1]:
import os
import time
from pprint import pprint

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
    
from catanatron_gym.envs.catanatron_env import ACTION_SPACE_SIZE
from catanatron_gym.features import get_feature_ordering

2022-01-15 10:56:32.729748: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-15 10:56:32.729783: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# Ensure datasets exists
import pathlib
import urllib.request

DATA_FOLDER = "./data"
DATASET_PATH = pathlib.Path(DATA_FOLDER, "1v1-ab2s-nodiscard", "main.csv.gzip")
VALIDATION_DATASET_PATH = pathlib.Path(DATA_FOLDER, "1v1-ab2s-nodiscard-validation", "main.csv.gzip")

DATASET_PATH.parents[0].mkdir(parents=True, exist_ok=True)
VALIDATION_DATASET_PATH.parents[0].mkdir(parents=True, exist_ok=True)

if not DATASET_PATH.exists():
    urllib.request.urlretrieve("https://catanatron-public.s3.us-east-2.amazonaws.com/1v1-ab2s-nodiscard/main.csv.gzip", DATASET_PATH)
if not VALIDATION_DATASET_PATH.exists():
    urllib.request.urlretrieve("https://catanatron-public.s3.us-east-2.amazonaws.com/1v1-ab2s-nodiscard-validation/main.csv.gzip", VALIDATION_DATASET_PATH)

In [2]:
BATCH_SIZE = 128
SHUFFLE_SEED = 1

rows_per_bytes = 6345 / 1209734  # ~0.0052
size = os.path.getsize(DATASET_PATH)  # bytes
estimated_rows = int(size * rows_per_bytes)
print(estimated_rows)

1004976


In [3]:
data = pd.read_csv(DATASET_PATH, nrows=10, compression='gzip')
validation_data = pd.read_csv(VALIDATION_DATASET_PATH, nrows=10, compression='gzip')

assert (data.columns == validation_data.columns).all()
data

,F_BANK_BRICK,F_BANK_DEV_CARDS,F_BANK_ORE,F_BANK_SHEEP,F_BANK_WHEAT,F_BANK_WOOD,"F_EDGE(0, 1)_P0_ROAD","F_EDGE(0, 1)_P1_ROAD","F_EDGE(0, 20)_P0_ROAD","F_EDGE(0, 20)_P1_ROAD",...,BT_3692,BT_3693,BT_3694,BT_3695,ACTION,RETURN,DISCOUNTED_RETURN,TOURNAMENT_RETURN,DISCOUNTED_TOURNAMENT_RETURN,VICTORY_POINTS_RETURN
0,19.0,25.0,19.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,110,0.0,0.0,6.0,3.185433,5.962317
1,19.0,25.0,19.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,52,0.0,0.0,6.0,3.185433,5.962317
2,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,107,0.0,0.0,6.0,3.185433,5.962317
3,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,45,0.0,0.0,6.0,3.185433,5.962317
4,17.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,6.0,3.185433,5.962317
5,17.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289,0.0,0.0,6.0,3.185433,5.962317
6,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,6.0,3.185433,5.962317
7,19.0,25.0,19.0,18.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,50,0.0,0.0,6.0,3.185433,5.962317
8,19.0,25.0,19.0,18.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289,0.0,0.0,6.0,3.185433,5.962317
9,19.0,25.0,18.0,17.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,6.0,3.185433,5.962317


In [4]:
# ===== Read Dataset
INPUT_COLUMNS = list(filter(lambda x: x.startswith("F_"), data.columns))
# INPUT_COLUMNS = list(filter(lambda x: x.startswith("BT_"), data.columns))
LABEL_COLUMN = "ACTION"
INPUT_SHAPE = (len(INPUT_COLUMNS),)

dataset = tf.data.experimental.make_csv_dataset(
    str(DATASET_PATH),
    batch_size=BATCH_SIZE,
    num_epochs=1,
    label_name=LABEL_COLUMN,
    select_columns=INPUT_COLUMNS + [LABEL_COLUMN],
    compression_type="GZIP",
    shuffle=True,  # shuffle will shuffle at the element level. nice.
    shuffle_seed=1,
)
labels_dataset = tf.data.experimental.make_csv_dataset(
    str(DATASET_PATH),
    batch_size=BATCH_SIZE,
    num_epochs=1,
    label_name=LABEL_COLUMN,
    select_columns=[LABEL_COLUMN],
    compression_type="GZIP",
    shuffle=True,
    shuffle_seed=1,
)
validation_dataset = tf.data.experimental.make_csv_dataset(
    str(VALIDATION_DATASET_PATH),
    batch_size=BATCH_SIZE,
    num_epochs=1,
    label_name=LABEL_COLUMN,
    select_columns=INPUT_COLUMNS + [LABEL_COLUMN],
    compression_type="GZIP",
    shuffle=True,
    shuffle_seed=1,
)

def preprocess(batch, label):
    features = tf.stack(
        [tf.cast(tensor, tf.float32) 
         for feature_name, tensor in batch.items() 
         if feature_name in INPUT_COLUMNS
        ], axis=1
    )
    return features, tf.stack(label)

dataset = dataset.map(preprocess)
validation_dataset = validation_dataset.map(preprocess)

2022-01-15 10:56:35.795027: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-01-15 10:56:35.821854: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-15 10:56:35.821879: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bcollazo-Erazer-X510): /proc/driver/nvidia/version does not exist
2022-01-15 10:56:35.822543: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# ===== Build Class Weights
# def count(counts, batch):
#     features, labels = batch
#     for i in range(ACTION_SPACE_SIZE):
#         class_i = labels == i
#         class_i = tf.cast(class_i, tf.int32)
#         counts[f"class_{i}"] += tf.reduce_sum(class_i)
#     return counts

# # This takes abount 3mins
# start = time.time()
# initial_state = {f"class_{i}": 0 for i in range(ACTION_SPACE_SIZE)}
# counts = labels_dataset.reduce(initial_state=initial_state, reduce_func=count)  # { "class_0": tf.Tensor, ... }
# print("Computing class counts took:", time.time() - start, "seconds")

2022-01-15 10:56:40.034404: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-15 10:56:40.080440: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3492135000 Hz


Computing class counts took: 161.37691235542297 seconds


In [10]:
# counts_array = np.array([counts[f"class_{i}"].numpy() for i in range(ACTION_SPACE_SIZE)]).astype(np.float32)

# fractions = counts_array/counts_array.sum()
# weights = (1 / counts_array) * (counts_array.sum() / ACTION_SPACE_SIZE)
# weights[weights == np.inf] = -np.inf  # Drop inf to min value (to sort of "clip" importance)
# weights[weights == -np.inf] = np.max(weights)  # Drop inf to min value (to sort of "clip" importance)
# class_weight = {i: weights[i] for i in range(ACTION_SPACE_SIZE)}
# pprint(class_weight)

# {0: 0.010215329, then many in 0.5-2 range, 289: 0.010377976}

# === Rejection Resampling
# def class_func(features, label):
#     return label
# n = ACTION_SPACE_SIZE
# target_dist = [1.0 / n for i in range(n)]
# resampler = tf.data.experimental.rejection_resample(
#     class_func, target_dist=target_dist)

# dataset = dataset.unbatch().apply(resampler).batch(BATCH_SIZE)\
#     .map(lambda extra_label, features_and_label: features_and_label)
# dataset

{0: 0.010215329,
 1: 0.70093286,
 2: 0.63069355,
 3: 0.6780919,
 4: 0.6585109,
 5: 0.69551516,
 6: 0.67753994,
 7: 0.7052385,
 8: 1.6360841,
 9: 1.5638474,
 10: 2.2866974,
 11: 1.429554,
 12: 2.3105004,
 13: 1.6022286,
 14: 2.488364,
 15: 1.7313734,
 16: 2.3713896,
 17: 1.5406901,
 18: 2.359625,
 19: 2.099263,
 20: 3329.4312,
 21: 2.2093108,
 22: 2.0313795,
 23: 1.7919436,
 24: 2.011741,
 25: 1.8981934,
 26: 2.1976445,
 27: 1.8263474,
 28: 2.1605654,
 29: 1.9596416,
 30: 2.1287923,
 31: 1.8333871,
 32: 1.8425187,
 33: 1.2578131,
 34: 1.2437173,
 35: 1.2601935,
 36: 1.8992763,
 37: 1.2249563,
 38: 1.6006881,
 39: 1.1580629,
 40: 1.105022,
 41: 1.7569557,
 42: 1.1785597,
 43: 1.5786777,
 44: 1.1810682,
 45: 1.0664418,
 46: 1.6772953,
 47: 1.2597166,
 48: 1.7709739,
 49: 1.2071904,
 50: 1.2423251,
 51: 1.2493175,
 52: 1.2050059,
 53: 1.6123153,
 54: 1.789055,
 55: 1.1719223,
 56: 1.2168974,
 57: 1.596084,
 58: 1.2186791,
 59: 1.6312743,
 60: 1.174817,
 61: 1.6523232,
 62: 1.5343001,
 63: 

/tmp/ipykernel_215136/1738080328.py:4: RuntimeWarning: divide by zero encountered in true_divide
  weights = (1 / counts_array) * (counts_array.sum() / ACTION_SPACE_SIZE)


In [11]:
# Things to try: Dropout. Less Learning Rate. Simpler Models. Validation Graph.
LAYERS = [16]

inputs = tf.keras.Input(shape=INPUT_SHAPE)

last_layer = inputs
for num_neurons in LAYERS:
    last_layer = tf.keras.layers.Dense(num_neurons, activation="relu", dtype='float64')(last_layer)

last_layer = tf.keras.layers.Dropout(0.2)(last_layer)
output_dense = tf.keras.layers.Dense(ACTION_SPACE_SIZE, dtype='float64')(last_layer)
model = tf.keras.Model(inputs=inputs, outputs=output_dense)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['categorical_accuracy']
)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 614)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                9840      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 290)               4930      
Total params: 14,770
Trainable params: 14,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Shuffle at the individual element level
# SHUFFLE_BUFFER_SIZE = 20000  # each game is like 200 samples...
# dataset = dataset.unbatch().shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# validation_dataset = validation_dataset.unbatch().shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
class_weight = {i: 1 for i in range(290)}
class_weight[0] = 0.01
class_weight[289] = 0.01

history = model.fit(
    dataset, 
    epochs=10,
    # steps_per_epoch=10,
    # validation_steps=10,
    validation_data=validation_dataset,
    class_weight=class_weight
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/10
7544/7544 [==============================] - 914s 121ms/step - loss: 1.6939 - categorical_accuracy: 0.0111 - val_loss: 4.5561 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
5036/7544 [===================>..........] - ETA: 5:15 - loss: 1.6010 - categorical_accuracy: 0.0000e+00

In [ ]:
# summarize history for categorical_accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model categorical_accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save("data/models/ab2-copycat-model")

In [ ]:
from pprint import pprint
import random
import numpy as np
import tensorflow as tf

from catanatron.game import Game
from catanatron.models.player import Player, RandomPlayer, Color
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron_experimental.play import play_batch
from catanatron_gym.envs.catanatron_env import to_action_space, ACTION_SPACE_SIZE, from_action_space
from catanatron_gym.features import create_sample_vector, create_sample


class MyPlayer(Player):
    def decide(self, game, playable_actions):
        """Should return one of the playable_actions.

        Args:
            game (Game): complete game state. read-only.
            playable_actions (Iterable[Action]): options to choose from
        Return:
            action (Action): Chosen element of playable_actions
        """
        # ===== YOUR CODE HERE =====
        action_ints = [to_action_space(a) for a in playable_actions]
        mask = np.zeros(ACTION_SPACE_SIZE, dtype=np.float)
        mask[action_ints] = 1
        mask[mask == 0] = np.nan

        # Get action probabilities with neural network.
        vector = create_sample_vector(game, self.color)
        record = create_sample(game, self.color)
        keys = [name[2:] for name in INPUT_COLUMNS] # remove the F_ prefix
        vector = [record[x] for x in keys]
        X = [vector]
        result = model.call(tf.convert_to_tensor(X))

        # Multiply mask with output, and take max.
        clipped_probabilities = np.multiply(mask, result[0])
        clipped_probabilities[np.isnan(clipped_probabilities)] = -np.inf
        action_index = np.argmax(clipped_probabilities)
        action = from_action_space(action_index, playable_actions)
        
        # print(result)
        # print("Playing", action)
        
        return action
        # ===== END YOUR CODE =====

# Play a simple 4v4 game. Edit MyPlayer with your logic!
players = [
    MyPlayer(Color.RED),
    RandomPlayer(Color.WHITE),
]
wins, results_by_player, games = play_batch(5, players)

pprint(wins)
pprint(results_by_player)

# Results: [64, 32, 32] with LR 1e-4 made a bot that won 60% of games against Random. Had %60 categorical accuracy.